# VALIDATION ALL — EDR (QNM, Rotation Curves, Lensing, Clusters)

Notebook maestro para validación numérica de la Teoría del Espacio Dinámico Rotacional (EDR).
Incluye: QNM (modo ℓ=2), curvas de rotación (SPARC/sintético), lente gravitacional (toy), y pruebas en cúmulos.

Requisitos: haber añadido el paquete `src/` con los módulos `edr_*` (creados en el repo EDR-Validation).

Autor: Camilo Robinson


## A) Setup: imports y comprobación de módulos

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.optimize import minimize, minimize_scalar

# Intentamos importar los módulos del paquete src (edr_*). Si no están, avisamos.
try:
    from edr_potential import deltaV as deltaV_basic
    from edr_deltaV import deltaV as deltaV_full
    from edr_alpha import compute_alpha_flow
    from edr_kerr_geometry import r_plus
    from edr_modes import psi0, normalize_psi
    print('Módulos edr_* importados correctamente')
except Exception as e:
    print('Atención: no se pudieron importar todos los módulos edr_*. Revisa src/ en el repo.')
    print('Error:', e)


-----
## B) QNM — cálculo de δω y α_flow (modo ℓ=2)
Procedimiento:
1. Construimos una aproximación del modo base Psi_0(r) centrado en el máximo del potencial de Regge–Wheeler.
2. Definimos una perturbación δV(r) (usando la función `deltaV` del paquete).
3. Evaluamos la integral perturbativa: δω = (1 / (2 ω0)) ∫ ψ0 δV ψ0 dr
4. Calculamos α_flow = δω / ω0
-----

In [ ]:
# Definiciones locales (Regge-Wheeler toy model)
def V_RW(r, l=2, M=1.0):
    return (1 - 2*M/r) * (l*(l+1)/r**2 - 6*M/r**3)

def find_rmax():
    f = lambda r: -V_RW(r)
    sol = minimize_scalar(f, bounds=(2.2, 12), method='bounded')
    return sol.x

r0 = find_rmax()
V0 = V_RW(r0)
r0, V0

In [ ]:
def omega_WKB3(r0):
    # WKB3 toy: derivada segunda numérica
    deriv = 1e-4
    Vpp = (V_RW(r0+deriv) - 2*V_RW(r0) + V_RW(r0-deriv)) / deriv**2
    n = 0
    w = np.sqrt(V0 - 1j*(n+0.5)*np.sqrt(-2*Vpp))
    return w

omega0 = omega_WKB3(r0)
omega0

In [ ]:
# Construir la malla radial y la psi0 aproximada
r = np.linspace(r0-10, r0+40, 5000)
Psi = np.exp(-((r - r0)**2) / 3.0)
Psi = Psi * (r/r0)**2
Psi = Psi / np.sqrt(simps(Psi**2, r))

plt.figure(figsize=(6,3))
plt.plot(r, Psi)
plt.title('Modo base aproximado Ψ₀(r)')
plt.xlabel('r')
plt.grid(True)
plt.show()

In [ ]:
# Intentamos obtener δV desde edr_deltaV (si existe), si no usamos deltaV_basic
try:
    dV = deltaV_full(r, r_plus(1, 0.7))
except Exception:
    dV = deltaV_basic(r, r0) if 'deltaV_basic' in globals() else 0.02 * np.exp(- (r - r0)/10)

plt.plot(r, dV)
plt.title('\u03b4V(r) (EDR) — usada en integral')
plt.xlabel('r')
plt.grid(True)
plt.show()

In [ ]:
integrand = Psi * dV * Psi
I = simps(integrand, r)
delta_omega = I / (2 * omega0)
alpha_flow = delta_omega / omega0

print('omega0 (geom) =', omega0)
print('delta_omega   =', delta_omega)
print('alpha_flow    =', alpha_flow)


-----
## C) Curvas de rotación (SPARC / sintético)
1. Si tienes la carpeta `data/sparc/` con archivos, el notebook leerá la primera galaxia.
2. Si no, genera datos sintéticos.
3. Ajusta parámetros EDR: `k_flow, eta, Omega0, R_Omega` por mínimos cuadrados.
-----

In [ ]:
from math import pi
def Sigma(r, Sigma0=500, Rd=3.0):
    return Sigma0 * np.exp(-r / Rd)

def M_enc(r):
    rs = np.linspace(1e-3, r, 600)
    integrand = 2 * pi * rs * Sigma(rs)
    return simps(integrand, rs)

def v_GR(r):
    G = 4.30091e-6
    return np.sqrt(G * M_enc(r) / np.maximum(r,1e-6))

In [ ]:
def Omega_flow(r, Omega0, R_Omega):
    return Omega0 * np.exp(-r / R_Omega)

def a_EDR(r, kflow, eta, Omega0, R_Omega):
    return kflow * eta * Omega_flow(r, Omega0, R_Omega)**2

def v_EDR(r, kflow, eta, Omega0, R_Omega):
    return np.sqrt(v_GR(r)**2 + r * a_EDR(r, kflow, eta, Omega0, R_Omega))

In [ ]:
# Cargar datos SPARC si existen
sparc_dir = 'data/sparc'
if os.path.isdir(sparc_dir) and len(os.listdir(sparc_dir))>0:
    fname = os.path.join(sparc_dir, os.listdir(sparc_dir)[0])
    data = np.loadtxt(fname)
    r_data = data[:,0]
    v_obs = data[:,1]
    print('Datos SPARC cargados de', fname)
else:
    # Generamos datos sintéticos
    r_data = np.linspace(0.5, 20, 40)
    v0 = np.array([v_GR(rr) for rr in r_data])
    v_obs = np.sqrt(v0**2 + 80**2) + np.random.normal(0,4,len(r_data))
    print('Generados datos sintéticos')


In [ ]:
def chi2(params):
    kflow, eta, Omega0, R_Omega = params
    v_model = v_EDR(r_data, kflow, eta, Omega0, R_Omega)
    return np.sum((v_model - v_obs)**2)

initial_guess = [0.01, 0.02, 0.1, 5.0]
sol = minimize(chi2, initial_guess, method='Nelder-Mead')
sol.x

In [ ]:
kflow, eta_opt, Omega0_opt, R_O_opt = sol.x
r_plot = np.linspace(0.5, 20, 300)
v_gr_plot = np.array([v_GR(rr) for rr in r_plot])
v_edr_plot = v_EDR(r_plot, kflow, eta_opt, Omega0_opt, R_O_opt)

plt.figure(figsize=(8,5))
plt.scatter(r_data, v_obs, label='Observado', color='black')
plt.plot(r_plot, v_gr_plot, label='Newton/GR', linestyle='--')
plt.plot(r_plot, v_edr_plot, label='EDR ajustado', linewidth=2)
plt.xlabel('r (kpc)')
plt.ylabel('v (km/s)')
plt.legend()
plt.title('Validación EDR — Curvas de Rotación')
plt.grid(True)
plt.show()

-----
## D) Lensing — toy model de anillo de Einstein
Calculamos el ángulo de deflexión en aproximación de lente débil y comparamos el radio de Einstein con/ sin corrección EDR (toy).
-----

In [ ]:
def alpha_deflection(M_proj, b):
    # ángulo de deflexión ~ 4GM/(c^2 b) en unidades geométricas adaptadas
    G = 4.30091e-6
    return 4 * G * M_proj / (b)

def M_projected(r):
    # masa proyectada toy (integral superficial hasta r)
    return M_enc(r)

# Elegimos parámetros toy
b_vals = np.linspace(0.5, 20, 100)
alpha_GR = np.array([alpha_deflection(M_projected(b), b) for b in b_vals])

# Corrección EDR (toy): añadimos term proporcional a deltaV en el proyectado
dV_proj = np.interp(b_vals, r, dV) if 'dV' in globals() else 0.01*np.exp(-b_vals/10)
alpha_EDR = alpha_GR * (1 + 0.5 * dV_proj)

plt.figure(figsize=(6,4))
plt.plot(b_vals, alpha_GR, label='GR')
plt.plot(b_vals, alpha_EDR, label='EDR (toy)')
plt.xlabel('impact parameter b (kpc)')
plt.ylabel('Deflection angle (rad, geo units)')
plt.legend()
plt.grid(True)
plt.title('Deflection angle: GR vs EDR (toy)')
plt.show()

-----
## E) Cúmulos — efecto en el perfil de masa proyectada y velocidad de dispersión (toy estimates)
Usamos un perfil NFW sencillo y añadimos la corrección EDR a la aceleración radial.
-----

In [ ]:
def rho_NFW(r, rho_s=0.1, r_s=1.0):
    return rho_s / ((r/r_s) * (1 + r/r_s)**2)

def M_NFW(r, rho_s=0.1, r_s=1.0):
    rs = np.linspace(1e-3, r, 800)
    integrand = 4 * np.pi * rs**2 * rho_NFW(rs, rho_s, r_s)
    return simps(integrand, rs)

r_vals = np.linspace(0.1, 5, 200)
Mproj = np.array([M_NFW(rv) for rv in r_vals])

# velocidad de dispersión toy ~ sqrt(G M / r)
v_disp_GR = np.sqrt(4.30091e-6 * Mproj / r_vals)

# añadir corrección EDR en aceleración radial: a_edr ~ deltaV_local
dV_local = np.interp(r_vals, r, dV) if 'dV' in globals() else 0.01*np.exp(-r_vals/10)
v_disp_EDR = np.sqrt(v_disp_GR**2 + r_vals * dV_local)

plt.figure(figsize=(6,4))
plt.plot(r_vals, v_disp_GR, label='GR (NFW)')
plt.plot(r_vals, v_disp_EDR, label='EDR corrected')
plt.xlabel('r (Mpc scaled)')
plt.ylabel('v_disp (km/s)')
plt.legend()
plt.grid(True)
plt.title('Velocidad de dispersión — Cúmulos (toy)')
plt.show()

-----
## F) Guardar resultados y resumen
Guardamos los parámetros ajustados y los resultados numéricos básicos en `results/`.
-----

In [ ]:
os.makedirs('results', exist_ok=True)
results = {
    'qnm': {
        'omega0': complex(omega0).real if 'omega0' in globals() else None,
        'delta_omega': complex(delta_omega).real if 'delta_omega' in globals() else None,
        'alpha_flow': complex(alpha_flow).real if 'alpha_flow' in globals() else None
    },
    'rotation_fit': {
        'kflow': float(kflow) if 'kflow' in globals() else None,
        'eta': float(eta_opt) if 'eta_opt' in globals() else None,
        'Omega0': float(Omega0_opt) if 'Omega0_opt' in globals() else None,
        'R_Omega': float(R_O_opt) if 'R_O_opt' in globals() else None
    }
}

with open('results/validation_summary.json', 'w') as f:
    json.dump(results, f, indent=2)

print('Resultados guardados en results/validation_summary.json')
results

### Notas finales
- Este notebook es un *framework* — para una validación completa debes reemplazar las piezas `toy` por las expresiones exactas derivadas en los apéndices (p. ej. proyección angular para δV, modo Leaver para ψ₀).
- Cuando pegues en GitHub, asegúrate de que `src/` contenga los módulos `edr_*` y que `data/sparc/` contenga archivos si deseas usar datos reales.
- Si quieres, lo compilo y te doy el archivo `.ipynb` listo para subir.
